<a href="https://colab.research.google.com/github/anespart1/educationuse/blob/main/Geoapify%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%A7%80%EC%98%A4%EC%BD%94%EB%94%A9%EA%B3%BC_%EC%A7%80%EB%8F%84%ED%99%94_20210830.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#공간네트워크 시각화1(feat. Geoapify Geocoding API 서비스)

In [31]:
import requests
import json
from requests.structures import CaseInsensitiveDict
import urllib
import folium
from folium.plugins import *
import pandas as pd

##이름이나 주소값으로 위치값 알아내기(지오코딩, Geocoding)

### 사용된 코드(숨김)

In [21]:
#Geoapify API 서비스를 사용해 지오코딩 하기(느림)
def Geoapify_geocoding(data=None, search_rowname=None, alam_off='y', report='y', apikey='d079ec0e5b1a4c3d9b4301d3618abfb4'):
  # 경고(SettingWithCopyWarning 발생, 기본 값)를 켜고 끄는 기능
  if alam_off != 'y':
    pd.set_option('mode.chained_assignment', 'warn') # SettingWithCopyWarning
  else:
    pd.set_option('mode.chained_assignment',  None) # <==== 경고를 끈다
  #사전 준비
  names, lats, lons, addresses, postcodes, counties, country_codes, result_types, sources, match_types = [], [], [], [], [], [], [], [], [], []
  for num, each in enumerate(data[search_rowname]):
    
    url = "https://api.geoapify.com/v1/geocode/search?text={}&apiKey={}".format(each, apikey)
    headers = CaseInsensitiveDict()
    headers["Accept"] = "application/json"
    resp = requests.get(url, headers=headers)
    statcode = resp.status_code
    resp_data = resp.json()
    
    if statcode == 200 and len(resp_data['features']) > 0:
      re_data = resp_data['features'][0]['properties']
      try:
        names.append(re_data['name'])
      except:
        names.append('')
      try:
        lats.append(re_data['lat'])
      except:
        lats.append(0.0)
      try:
        lons.append(re_data['lon'])
      except:
        lons.append(0.0)
      try:
        addresses.append(re_data['address_line2'])
      except:
        addresses.append('')
      try:
        postcodes.append(re_data['address_line2'])
      except:
        postcodes.append('')
      try:
        counties.append(re_data['county'])
      except:
        counties.append('')
      try:
        country_codes.append(re_data['country_code'])
      except:
        country_codes.append('')
      try:
        result_types.append(re_data['result_type'])
      except:
        result_types.append('')
      try:
        sources.append(re_data['datasource']['sourcename'])
      except:
        sources.append('')
      try:
        match_types.append(re_data['rank']['match_type'])
      except:
        match_types.append('')
    else:
      names.append('')
      lats.append(0.0)
      lons.append(0.0)
      addresses.append('')
      postcodes.append('')
      counties.append('')
      country_codes.append('')
      result_types.append('')
      sources.append('')
      match_types.append('')
      if report == 'y':
        print('"{}" 를 찾지 못했습니다'.format(each))
  
  data['검색결과'] = names
  data['Y'] = lats
  data['X'] = lons
  data['찾은주소'] = addresses
  data['우편번호'] = postcodes
  data['시도'] = counties
  data['국가코드'] = country_codes
  data['구분'] = result_types
  data['소스데이터'] = sources
  data['정확도'] = match_types
  return data

In [29]:
def spatial_visualizing(data=None, zoom= 7, xcor_name=None, ycor_name=None, 
                        minimap=True, measure=True, position=True, fullscrean=True):
  m = folium.Map(location=[data[ycor_name].mean(), data[xcor_name].mean()], zoom_start=zoom,
                 control_scale=True) #스케일바 추가
  #지도 타일 종류 추가
  for i in ['Stamen Terrain', 'Stamen Water Color', 'cartodbpositron']:
    folium.TileLayer(i).add_to(m)
  
  #Heatmap 추가
  heatmap = HeatMap(zip(data['Y'].values.tolist(), data['X'].values.tolist()),
                    name='Density', radius=22, min_opacity=0.28)
  m.add_child(heatmap)

  #미니맵 추가
  if minimap==True:
    m.add_child(MiniMap())

  #거리측량기 추가
  if measure==True:
    m.add_child(MeasureControl())

  #좌표기 추가
  if position==True:
    MousePosition(prefix='경위도 좌표:').add_to(m)

  #전체화면모드 추가
  if fullscrean==True:
    Fullscreen(title='전체화면', title_cancel='돌아가기', force_separate_button=True).add_to(m)

  return m

###코드 실행부(Ctrl+F10)

####데이터 지오코딩하기

In [22]:
#예시 데이터 가져오기 
url = 'https://raw.githubusercontent.com/anespart1/educationuse/master/소방청_시도_소방서_현황_20200121.csv'
link = urllib.parse.quote(url, safe=':/?-=') #URL에 포함된 한글 때문에 문제가 생기면 사용
df = pd.read_csv(link, encoding='EUC-KR')

#지오코딩 - 소방서 이름을 기준으로, 앞에서 10개만 뽑아서 실시(오래걸리기 때문)
ndf = Geoapify_geocoding(data=df.head(10), search_rowname='소방서', alam_off='y', report='n')
df1 = ndf[ndf.X != 0].copy() #지오코딩이 완료된 데이터
df2 = ndf[ndf.X == 0].copy() #지오코딩이 실패한 데이터
df3 = Geoapify_geocoding(data=df2, search_rowname='주소', alam_off='y') #실패데이터를 주소 기준으로 다시 지오코딩
concat_df = pd.concat([df1, df2], axis=0) #지오코딩이 완료된 데이터 + 다시 지오코딩한 데이터를 결합
concat_df.sort_values(by=['순번'], axis=0, inplace=True) #전체 데이터를 순번 열을 기준으로 정렬

####지오코딩한 데이터 시각화하기

In [32]:
map = spatial_visualizing(data=concat_df, zoom= 7, xcor_name='X', ycor_name='Y')

#마커 클러스터 및 클러스터 그룹 추가
marker_cluster = MarkerCluster(control=False)
map.add_child(marker_cluster)
group = folium.plugins.FeatureGroupSubGroup(marker_cluster, '소방서')
map.add_child(group)

#전국 소방서 추가
for n, station in concat_df.iterrows():
  popup = folium.Popup('위도: {}<br>경도: {}'.format(station['Y'], station['X']),
                       max_width=300) #max_width를 설정하면 세로로 길게 나오는 현상을 피할 수 있음
  folium.Marker(location=[station['Y'], station['X']], popup=popup, tooltip=station['소방서'],
                icon=folium.Icon(color="red", icon="fire-extinguisher", prefix='fa')).add_to(group)
#레이어 컨트롤러 추가
folium.LayerControl().add_to(map)
map

In [33]:
#지도 저장하기
map.save('map_save.html')
from google.colab import files
files.download('/content/map_save.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>